In [4]:
from bs4 import BeautifulSoup
import requests
import urllib.request
import os
import json
from io import StringIO
from pdfminer.high_level import extract_text_to_fp
from pdfminer.high_level import extract_pages
from pdfminer.layout import LAParams
from pdfminer.layout import LTTextContainer
from json import JSONEncoder
from collections import namedtuple



## Extracting Salgsoppgave PDFs from DNB Eiendom

In [3]:
## done with leiligheter
## html_doc = open("./DNB/Boliger til salgs _ DNB Eiendom.html")

## eneboliger
html_doc = open("./DNB/Boliger til salgs _ DNB Eiendom_Enebolig.html")

## rekkehus
#html_doc = open("./DNB/Boliger til salgs _ DNB EiendomRekkehus.html")

## tomansbolig
##html_doc = open("./DNB/Boliger til salgs _ DNB EiendomTomansbolig.html")
soup = BeautifulSoup(html_doc, 'html.parser')

In [4]:
## extract bolig links
bolig_links = []
for link in soup.find_all('a'):
    url = link.get('href')
    if "/bolig/" in url:
        bolig_links.append(url)

print(len(bolig_links))

571


In [5]:
## download html and extract pdf links
pdf_links = []
for bolig_link in bolig_links:
    r = requests.get(bolig_link)
    soup = BeautifulSoup(r.text, 'html.parser')
    for link in soup.find_all('a'):
        url = link.get('href')
        if ".pdf" in url:
            pdf_links.append(url)
            
##should be as many as from the line before
print(len(pdf_links))
print(pdf_links[0])
print(pdf_links[100])

473
https://dnbeiendom.no/api/v1/properties/c2a43248-65b1-4a30-25a2-08d9623795d1/documents/c2a43248-65b1-4a30-25a2-08d9623795d1.pdf
https://dnbeiendom.no/api/v1/properties/be7e8a28-5557-410d-ec1d-08da4fcaee2a/documents/be7e8a28-5557-410d-ec1d-08da4fcaee2a.pdf


In [7]:
## Save PDFs in the folder function
#leiligheter
#folderUrl = "./DNB/PDFSLeilighet/"

#eneboliger
folderUrl = "./DNB/PDFSEnebolig/"

#rekkehus
#folderUrl = "./DNB/PDFSRekkehus/"

#Tomansbolig
#folderUrl = "./DNB/PDFSTomansbolig/"

def download_file(download_url, filename):
    if os.path.exists(folderUrl + filename + ".pdf"):
        print("exists")
        return
    response = urllib.request.urlopen(download_url)    
    file = open(folderUrl + filename + ".pdf", 'wb')
    file.write(response.read())
    file.close()
    print(f"made file {filename}")

In [8]:
for pdf_link in pdf_links:
    try:
        name = pdf_link.split("/")[-1].split(".")[0]
        download_file(pdf_link,name)
    except:
        print(f"file {name} failed")

made file c2a43248-65b1-4a30-25a2-08d9623795d1
made file b8232b69-8c0f-461e-49cc-08da5d9a69a5
made file ba6c4246-fb81-4831-49c6-08da5d9a69a5
made file e2256337-564d-483e-fc8d-08da59990ca9
made file 467188e0-24a3-4b34-6ebf-08da29b50bca
made file aa1806d3-39ac-43f0-e7b4-08da5cd52437
made file 56ce5179-9824-4ada-28ff-08da5ce03283
made file f1fcb3e0-8353-49b4-e7b3-08da5cd52437
made file 44614faa-ab51-4d3a-240b-08d850a20bd1
made file 4e99e44b-e2e1-47da-61c1-08da5bf87805
made file 427a8cb4-0841-427c-61bf-08da5bf87805
made file 3a3722ab-02e9-4e21-0179-08da5b2bf222
made file db545e96-24ba-4f95-67c8-08da5b2f6431
made file 980e3470-bc00-481b-016d-08da5b2bf222
made file ad1a4c0d-56c9-4d52-016b-08da5b2bf222
made file 0a7530aa-ddee-4e13-0169-08da5b2bf222
made file 57f24009-44bc-4b1c-0167-08da5b2bf222
made file b36abfbb-b71f-45d9-a733-08da5b3ef454
made file a4ddede2-ef52-4fdf-015d-08da5b2bf222
made file 6ff80be3-0670-4373-b8c7-08d9f857dc14
made file 0672c133-a025-470f-c2f2-08da5a6bbd2d
made file 3b9

In [9]:
number_of_files = len(os.listdir(folderUrl))
print(number_of_files)

471


In [2]:
class Bolig:
        prisantydning = []
        byggeaar = []
        bruksareal = []
        number_of_rooms = []
        heating = []
        location = []
        energy_label = []
        property_type_description = []

In [5]:
##  Extracting info without Azure?
#leiligheter
#folderUrl = "./DNB/PDFSLeilighet/"
#folderOutputUrl = "./DNB/JSON/Untreated/Leilighet/"

#eneboliger
folderUrl = "./DNB/PDFSEnebolig/"
folderOutputUrl = "./DNB/JSON/Untreated/Enebolig/"

#rekkehus
#folderUrl = "./DNB/PDFSTomansbolig/"
#folderOutputUrl = "./DNB/JSON/Untreated/Tomansbolig/"

i = 0
for file in os.listdir(folderUrl):
    i+=1
    print(f"file number {i} {file}")
    if os.path.exists(folderOutputUrl + file.split(".")[0] +".json"):
            print("exists")
            continue
    with open(folderUrl + file, 'rb') as f:
        bolig = Bolig()
        bolig.prisantydning = []
        bolig.byggeaar = []
        bolig.bruksareal = []
        bolig.number_of_rooms = []
        bolig.heating = []
        bolig.location = []
        bolig.energy_label = []
        bolig.property_type_description = []
        bolig.property_type_description.append('enebolig')
        try:
            for page_layout in extract_pages(f):
                for element in page_layout:
                    if isinstance(element, LTTextContainer):
                        # vi trenger prisantydning
                        if 'prisantydning' in str.lower(element.get_text()):
                            bolig.prisantydning.append(str.lower(element.get_text()).split('\n'))
                        if 'byggeår' in str.lower(element.get_text()):
                            bolig.byggeaar.append(str.lower(element.get_text()).split('\n'))
                        if 'bruksareal' in str.lower(element.get_text()):
                            bolig.bruksareal.append(str.lower(element.get_text()).split('\n'))
                        if 'antall rom' in str.lower(element.get_text()):
                            bolig.number_of_rooms.append(str.lower(element.get_text()).split('\n'))
                        if 'oppvarming' in str.lower(element.get_text()):
                            bolig.heating.append(str.lower(element.get_text()).split('\n'))
                        if 'adresse' or 'kommune' in str.lower(element.get_text()):
                            bolig.location.append(str.lower(element.get_text()).split('\n'))
                        if 'energikarakter' or 'energifarge' in str.lower(element.get_text()):
                            bolig.energy_label.append(str.lower(element.get_text()).split('\n'))
        except:
            print(f"could not extract from document number {i}")
        
        jsonStr = json.dumps(bolig.__dict__,indent=4,sort_keys=True,ensure_ascii=False)
        with open(folderOutputUrl + file.split(".")[0] +".json", 'w', encoding="utf-8") as outfile:
                outfile.write(jsonStr)
                outfile.close()
    

file number 1 00981054-a42d-4ef3-7ab3-08da43067f37.pdf
exists
file number 2 01374af2-6fbf-4285-bdf5-08da363ee68e.pdf
exists
file number 3 0140924c-c211-4e4e-ecfc-08da4e382ae0.pdf
exists
file number 4 024c2c2d-d30d-4c57-082e-08da55bcfa32.pdf
exists
file number 5 0259f397-5570-4d27-3f2a-08da34647a19.pdf
exists
file number 6 029fee0b-31cc-44ba-ffa9-08da43e942d2.pdf
exists
file number 7 0327a59a-e15f-41ba-a17b-08da1931067c.pdf
exists
file number 8 037035a0-4816-424a-4b94-08da17a0bb80.pdf
exists
file number 9 04a669ed-2694-4997-cf25-08da541faf08.pdf
exists
file number 10 04a80745-9ddb-46a9-0de3-08da40a257e4.pdf
exists
file number 11 0525f314-7ea6-4317-3885-08da0ea7ed29.pdf
exists
file number 12 05c9917e-fc67-40d1-8f09-08da42140ef8.pdf
exists
file number 13 0672c133-a025-470f-c2f2-08da5a6bbd2d.pdf
exists
file number 14 083365c0-9159-497c-4e12-08da4a48a171.pdf
exists
file number 15 0853fbc8-0d69-44e4-baa5-08d7ce3d0487.pdf
exists
file number 16 08adee8c-2e17-445d-8aa8-08da2dab9392.pdf
exists
f

In [6]:
number_of_files = len(os.listdir(folderOutputUrl))
print(number_of_files)

471
